<a href="https://colab.research.google.com/github/victorsa2/sales_meli/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparando o Ambiente no Github

In [ ]:
!git clone https://github.com/victorsa2/sales_meli.git

Cloning into 'sales_meli'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


## Subindo o arquivo .env com as variáveis no drive

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env (1)


##Instalando Dependencias não disponíveis no colab

In [24]:
pip install python-dotenv

In [25]:
import requests
import webbrowser
from dotenv import load_dotenv
import os

##Consultando API do Mercado Livre

#Obter Access Token e User ID

In [29]:
load_dotenv('/content/.env')

APP_ID = os.getenv("APP_ID")
APP_SECRET = os.getenv("APP_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")

def get_access_token_and_user_id():

    auth_url = f'https://auth.mercadolivre.com.br/authorization?response_type=code&client_id={APP_ID}&redirect_uri={REDIRECT_URI}'
    return auth_url

def request_token(authorization_code):
    token_url = 'https://api.mercadolibre.com/oauth/token'
    payload = {
        'grant_type': 'authorization_code',
        'client_id': os.getenv("APP_ID"),
        'client_secret': os.getenv("APP_SECRET"),
        'code': authorization_code,
        'redirect_uri': os.getenv("REDIRECT_URI")
    }

    response = requests.post(token_url, data=payload)
    token_data = response.json()

    access_token = token_data['access_token']
    user_id = token_data['user_id']

    return access_token, user_id

if __name__ == "__main__":
    auth_url = get_access_token_and_user_id()
    print("Abra o seguinte URL no seu navegador para obter o código de autorização:")
    print(auth_url)

    authorization_code = input("Insira o código de autorização: ")
    access_token, user_id = request_token(authorization_code)


Abra o seguinte URL no seu navegador para obter o código de autorização:
https://auth.mercadolivre.com.br/authorization?response_type=code&client_id=5500572046072825&redirect_uri=https://victorsa.netlify.app
Insira o código de autorização: TG-66fae1297b02b00001f07138-478295390


#Obter dimensão de Categorias

In [30]:
import requests
import pandas as pd

def fetch_categories():
    url = "https://api.mercadolibre.com/sites/MLB/categories"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao obter categorias: {response.status_code}")
        return []

categories_data = fetch_categories()
categories_list = []

for category in categories_data:
    categories_list.append({
        'id_category': category['id'],
        'name': category['name'],
    })

dim_Category = pd.DataFrame(categories_list)
dim_Category.to_csv('teste.csv')

print("Dimensão Categorias:")
print(dim_Category.head())

Dimensão Categorias:
  id_category                      name
0     MLB5672  Acessórios para Veículos
1   MLB271599                      Agro
2     MLB1403       Alimentos e Bebidas
3     MLB1071                   Animais
4     MLB1367   Antiguidades e Coleções


# Consumindo API por produtos para conseguir alguns vendedores de exemplo, pesquisando por todas as categorias do Mercado Livre.


In [48]:
def fetch_categories():
    url = "https://api.mercadolibre.com/sites/MLB/categories"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter categorias: {e}")
        return []

def fetch_sales_in_category(category_id, access_token, site_id):
    url = f'https://api.mercadolibre.com/sites/{site_id}/search?category={category_id}'
    headers = {'Authorization': f'Bearer {access_token}'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()['results']
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter vendas na categoria {category_id}: {e}")
        return []

ACCESS_TOKEN = access_token
SITE_ID = 'MLB'

sales_list = []
categories_data = fetch_categories()

for category in categories_data:
    category_id = category['id']
    print(f"\nBuscando vendas na categoria: {category['name']} (ID: {category_id})")

    sales_data = fetch_sales_in_category(category_id, ACCESS_TOKEN, SITE_ID)

    if sales_data:
        for item in sales_data:
            if 'seller' in item and 'nickname' in item['seller']:
                sales_list.append({
                    'seller_id': item['seller']['id'],
                    'seller_nickname': item['seller']['nickname'],
                    'category_id': category['id'],
                    'category': category['name'],
                    'item_id': item['id'],
                    'item_name': item['title'],
                    'currency': item['currency_id'],
                    'price': item['price']
                })
    else:
        print("Nenhum resultado encontrado para esta categoria.")

df_sales = pd.DataFrame(sales_list)
df_sales.to_csv('produtos.csv',encoding='utf-8-sig')

print(df_sales)


Buscando vendas na categoria: Acessórios para Veículos (ID: MLB5672)

Buscando vendas na categoria: Agro (ID: MLB271599)

Buscando vendas na categoria: Alimentos e Bebidas (ID: MLB1403)

Buscando vendas na categoria: Animais (ID: MLB1071)

Buscando vendas na categoria: Antiguidades e Coleções (ID: MLB1367)

Buscando vendas na categoria: Arte, Papelaria e Armarinho (ID: MLB1368)

Buscando vendas na categoria: Bebês (ID: MLB1384)

Buscando vendas na categoria: Beleza e Cuidado Pessoal (ID: MLB1246)

Buscando vendas na categoria: Brinquedos e Hobbies (ID: MLB1132)

Buscando vendas na categoria: Calçados, Roupas e Bolsas (ID: MLB1430)

Buscando vendas na categoria: Câmeras e Acessórios (ID: MLB1039)

Buscando vendas na categoria: Carros, Motos e Outros (ID: MLB1743)

Buscando vendas na categoria: Casa, Móveis e Decoração (ID: MLB1574)

Buscando vendas na categoria: Celulares e Telefones (ID: MLB1051)

Buscando vendas na categoria: Construção (ID: MLB1500)

Buscando vendas na categoria: El